In [1]:
from jenga.tasks.shoes import ShoeCategorizationTask
from jenga.corruptions.image import GaussianNoiseCorruption, MotionBlurCorruption, FogCorruption, BrightnessCorruption

from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

import matplotlib.pyplot as plt

import imgaug.augmenters as iaa

Using TensorFlow backend.


### Instantiate the shoe categorization task and train a baseline model

In [4]:
task = ShoeCategorizationTask(seed=42)

baseline_model = task.fit_baseline_model(task.train_data, task.train_labels)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/train-labels-idx1-ubyte.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1091)

### Train another model on data augmented with gaussian noise corruptions 

In [ ]:
import numpy as np

augmented_data = [task.train_data]
augmented_labels= [task.train_labels]

for severity in range(1, 6):
    indexes = np.random.choice(range(0, len(task.train_data)), 2400)
    sample = task.train_data[indexes, :, :].copy()
    corrupted_sample = GaussianNoiseCorruption(fraction=1.0, severity=severity).transform(sample)
    augmented_data.append(corrupted_sample)
    augmented_labels.append(task.train_labels[indexes].copy())

gaussian_noise_model = task.fit_baseline_model(np.concatenate(augmented_data), 
                                               np.concatenate(augmented_labels))


### Evaluate how well both models can handle corrupted test data

In [ ]:
evaluator = CorruptionImpactEvaluator(task)

corruptions  = []
for fraction in [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.99]:
    corruptions.append(GaussianNoiseCorruption(fraction=fraction, severity=4))
    
gaussian_noise_results = evaluator.evaluate(baseline_model, 10, *corruptions)        
gaussian_noise_results_aug = evaluator.evaluate(gaussian_noise_model, 10, *corruptions)    

### Train another model on data augmented with motion blur corruptions 

In [ ]:
import numpy as np

augmented_data = [task.train_data]
augmented_labels= [task.train_labels]

for severity in range(1, 6):
    indexes = np.random.choice(range(0, len(task.train_data)), 2400)
    sample = task.train_data[indexes, :, :].copy()
    corrupted_sample = MotionBlurCorruption(fraction=1.0, severity=severity).transform(sample)
    augmented_data.append(corrupted_sample)
    augmented_labels.append(task.train_labels[indexes].copy())

motion_blur_model = task.fit_baseline_model(np.concatenate(augmented_data), 
                                            np.concatenate(augmented_labels))

### Evaluate how well both models can handle corrupted test data

In [ ]:
evaluator = CorruptionImpactEvaluator(task)

corruptions  = []
for fraction in [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.99]:
    corruptions.append(MotionBlurCorruption(fraction=fraction, severity=4))
    
motion_blur_results = evaluator.evaluate(baseline_model, 10, *corruptions)        
motion_blur_results_aug = evaluator.evaluate(motion_blur_model, 10, *corruptions)    

### Train another model on data augmented with fog corruptions 

In [ ]:
import numpy as np

augmented_data = [task.train_data]
augmented_labels= [task.train_labels]

for severity in range(1, 6):
    indexes = np.random.choice(range(0, len(task.train_data)), 2400)
    sample = task.train_data[indexes, :, :].copy()
    corrupted_sample = FogCorruption(fraction=1.0, severity=severity).transform(sample)
    augmented_data.append(corrupted_sample)
    augmented_labels.append(task.train_labels[indexes].copy())

fog_model = task.fit_baseline_model(np.concatenate(augmented_data), 
                                    np.concatenate(augmented_labels))

### Evaluate how well both models can handle corrupted test data

In [ ]:
evaluator = CorruptionImpactEvaluator(task)

corruptions  = []
for fraction in [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.99]:
    corruptions.append(FogCorruption(fraction=fraction, severity=4))
    
fog_results = evaluator.evaluate(baseline_model, 10, *corruptions)        
fog_results_aug = evaluator.evaluate(fog_model, 10, *corruptions)   

### Plot the results

In [ ]:
def as_series(results):
    medians = []
    for result in results:
        medians.append(np.median(result.corrupted_scores))
    return medians

ax = plt.gca()        

fractions = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.99]
markersize=6

l = ax.axhline(gaussian_noise_results[0].baseline_score, linestyle='--', color='black')    

plt.plot(fractions, as_series(gaussian_noise_results_aug), color='#dc6082', linestyle='dotted', 
         marker='^', markersize=markersize, fillstyle='none')
plt.plot(fractions, as_series(gaussian_noise_results), color='#dc6082', 
         marker='o', markersize=markersize)

plt.plot(fractions, as_series(motion_blur_results_aug), color='#1e4052', linestyle='dotted', 
         marker='^', markersize=markersize, fillstyle='none')
plt.plot(fractions, as_series(motion_blur_results), color='#1e4052', 
         marker='o', markersize=markersize)

plt.plot(fractions, as_series(fog_results_aug), color='#e1a677', linestyle='dotted', 
         marker='^', markersize=markersize, fillstyle='none')
plt.plot(fractions, as_series(fog_results), color='#e1a677', 
         marker='o', markersize=markersize)


ax.set_ylim((0.9, 0.999))
ax.tick_params(axis='both', which='major', labelsize=22)
ax.tick_params(axis='both', which='minor', labelsize=22)    
ax.set_ylabel('AUC', fontsize=24)
ax.set_xlabel('fraction', fontsize=24)
    
ax.legend(['clean', 'noise (aug)', 'noise', 'motion (aug)', 'motion', 'fog (aug)', 'fog'], 
          fontsize=12)    
    
plt.tight_layout()
plt.gcf().set_size_inches(7, 5)

plt.show()